In [1]:
!pip install pandas transformers guidance numpy evaluate rouge-score nltk py-rouge accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB

In [2]:
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import json
import nltk
import numpy as np
from guidance import gen
from evaluate import load
import csv
import nltk.translate.bleu_score as bleu
from rouge import Rouge
from transformers import BertTokenizer, BertModel
import torch
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import warnings
from typing import Any, Dict, Tuple
import re

warnings.filterwarnings("ignore", category=UserWarning, message="A new version of the following files was downloaded from")

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Load Dataset

- id: unique identifier of the example
- source: original model input for paraphrase generation
- hyp1: first alternative paraphrase of the source
- hyp2: second alternative paraphrase of the source
- label: hyp1 or hyp2, based on which of those has been annotated as hallucination
- type: hallucination category assigned.
  - Possible values: addition, named-entity, number, conversion, date, tense, negation, gender, pronoun, antonym, natural

## Trial dataset

This is a small list of examples, provided to help the participants get familiar with the task. Each example contains the following fields:
- id
- source
- hyp1
- hyp2
- label
- type

In [ ]:
from datasets import load_dataset

#load the trial data for both English and Swedish
trial_ds = load_dataset("Eloquent/HalluciGen-PG", name="trial")

#load the trial data only for Swedish
trial_ds_sv = load_dataset("Eloquent/HalluciGen-PG", name="trial", split="trial_swedish")

print(trial_ds.keys())

trial_df_en = trial_ds['trial_english'].to_pandas()
trial_df_se= trial_ds['trial_swedish'].to_pandas()

print('English:')

display(trial_df_en.head())

print('Swedish')
display(trial_df_se.head())

dict_keys(['trial_swedish', 'trial_english'])
English:


,id,source,type,hyp1,hyp2,label
0,0,The population has declined in some 210 of the...,antonym,In the majority of Sweden's 280 municipalities...,In the majority of Sweden's 280 municipalities...,hyp1
1,1,The draft agenda as drawn up by the Conference...,negation,The Conference of Presidents hasn't distribute...,The Conference of Presidents has distributed t...,hyp1
2,2,I am always grateful for comments and suggesti...,antonym,I think I need to listen to everyone's views i...,I think I need to listen to everyone's views i...,hyp1
3,3,The fact is that a key omission from the propo...,named entity,Agenda 2030 does not include a chapter on rene...,Agenda 2000 does not include a chapter on rene...,hyp1
4,4,"We need quite specific legislative proposals, ...",natural,Legislative proposals based on the Treaty of A...,Legislative proposals based solely on the Trea...,hyp2


Swedish


,id,source,type,hyp1,hyp2,label
0,0,"Men intäkterna från mjukvarulicenser, ett mått...",number,"Intäkter från programvarulicenser, en metrik s...","Intäkter från programvarulicenser, en metrik s...",hyp2
1,1,Hongkong-universitet samarbetar med universite...,natural,University of Hong Kong samarbetar med olika k...,University of Hong Kong samarbetar med olika k...,hyp1
2,2,Län med befolkningsminskning kommer att vara V...,named entity,"Vermillion, Posey och Madison är län som komme...","Vermillion, Posey och Marion är län som kommer...",hyp2
3,3,Google presenterar en prototyp för en självkör...,addition,En prototyp för en självkörande bil presentera...,Enligt ett blogginlägg från Google har företag...,hyp2
4,4,Lagförslaget säger att en kvinna som genomgår ...,negation,Förslaget stadgar att en kvinna som genomgår e...,Förslaget stadgar att en kvinna som genomgår e...,hyp1


In [ ]:
print('Swedish')
row = trial_df_se.iloc[0]  # Selecting the first row
print("Source:", row['source'])
print("hyp1:", row['hyp1'])
print("hyp2:", row['hyp2'])


Swedish
Source: Men intäkterna från mjukvarulicenser, ett mått som finansanalytiker följer noga, minskade med 21 procent till 107,6 miljoner dollar.
hyp1: Intäkter från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 21 procent till ett belopp av 107,6 miljoner dollar.
hyp2: Intäkter från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 42 procent till ett belopp av 107,6 miljoner dollar.


In [ ]:
#load the test data for the generation step in both English and Swedish
data = load_dataset("Eloquent/HalluciGen-PG", "test_generation")


print(data.keys())

data_df_en = data['test_generation_english'].to_pandas()
data_df_se= data['test_generation_swedish'].to_pandas()


dict_keys(['test_generation_swedish', 'test_generation_english'])


In [ ]:
data_df_en

,id,source
0,0,It has enabled us to support and encourage an ...
1,1,"Therefore, I am calling for an increase in the..."
2,2,"In other words, a person may be prevented from..."
3,3,None of my 34 amendments were adopted and my a...
4,4,The final key feature of the report is its ori...
...,...,...
114,114,I am referring to the current vote but in rela...
115,115,When I became a Member of the European Parliam...
116,116,"Mr President, let me express my thanks for the..."
117,117,We can replace 16 out of 626 Members of the Eu...


In [ ]:
data_df_se

,id,source
0,0,Kvinnor kommer att möta högre bilförsäkringspr...
1,1,"Rörelseintäkterna var 1,45 miljarder dollar, e..."
2,2,"Mandela tillbaka på sjukhus i ""allvarligt men ..."
3,3,Egypten fryser Muslimska brödraskapets tillgån...
4,4,"P-pillret, som de kallar ""polypiller"", skulle ..."
...,...,...
71,71,Du kommer att komma på saker! Jag kommer att b...
72,72,Påven uppmanar till åtgärder mot klimatförändr...
73,73,Egyptens ledare inför undantagstillstånd i tre...
74,74,En svartvit bild av en katt som ligger på en m...


In [ ]:
# Replace "'" with '//"'
data_df_en['source'] = data_df_en['source'].str.replace("'", "\\'")

In [ ]:
selected_value = data_df_en.loc[12, 'source']
print(selected_value)

B4-0188/99 by Mrs d\'Ancona, on behalf of the PSE Group, on the death penalty against Greg Summers - Texas, USA; Leonard Peltier


In [ ]:
data_df_se['source'] = data_df_se['source'].str.replace("'", "\\'")

In [4]:
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.2 MB/s eta 0:00:00


In [5]:
!pip install -U "transformers==4.38.1" --upgrade
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 16.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.0
    Uninstalling transformers-4.40.0:
      Successfully uninstalled transformers-4.40.0


ERROR: Operation cancelled by user


In [1]:
#load model Gemma

from transformers import AutoTokenizer, pipeline
import torch

model = "google/gemma-7b-it"
model_id = "google/gemma-7b-it"
dtype = torch.bfloat16

# use quantized model
pipeline = pipeline(
    "text-generation",
    model=model,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True}
    },
)

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

# Prompt Version 1

In [2]:
def generate_paraphrase(source):
    # Generate the prompt

    answer_format = {"hyp+": "", "hyp-": ""}

    user_prompt = f'''
    <start_of_turn>user
    Given the src below, generate a paraphrase hypothesis  hyp+ that is supported by src and a second paraphrase hyp- that is not supported by src.

    Provide the result in the following format: {answer_format}

    Src: {source}
    <end_of_turn>

    Result:
    <start_of_turn>model
    '''

    messages = [
      #{"role": "user", "content": "You are a text generator. You are specialized in paraphrasing texts."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=150,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
    generated_paraphrase = outputs[0]["generated_text"][len(prompt):]

    return generated_paraphrase

In [8]:
def generate_paraphrase_se(source):
    # Generate the prompt

    answer_format = {"hyp+": "", "hyp-": ""}

    user_prompt = f'''
    <start_of_turn>användare
    Med tanke på källan nedan, generera en parafras-hypotes hyp+ som stöds av källan och en andra parafras hyp- som inte stöds av källan.

    Ge resultatet i följande format: {answer_format}

    Källa: {source}
    <end_of_turn>

    Resultat:
    <start_of_turn>modell
    '''

    messages = [
      #{"role": "user", "content": "You are a text generator. You are specialized in paraphrasing texts."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=150,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
    generated_paraphrase = outputs[0]["generated_text"][len(prompt):]

    return generated_paraphrase

In [ ]:
def generate_paraphrase_se(source):
    # Generate the prompt

    answer_format = {"hyp+": "", "hyp-": ""}

    user_prompt = f'''
      <start_of_turn>användare
      Som en AI-modell är din uppgift att generera två parafraser baserade på den angivna källtexten. Den första parafrasen, märkt som 'hyp+', ska stödjas av källtexten. Den andra parafrasen, märkt som 'hyp-', ska inte stödjas av källtexten.

      Här är ett exempel för att illustrera detta:
      Källa: Intäkterna från mjukvarulicenser, ett mått som finansanalytiker följer noga, minskade med 21 procent till 107,6 miljoner dollar.
      hyp+: Intäkter från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 21 procent till ett belopp av 107,6 miljoner dollar.
      Detta är en parafras som stöds av källan. Den säger i princip samma sak som källan: intäkterna har minskat i de flesta kommunerna. Formuleringen är annorlunda, men betydelsen är densamma. Därför märks den som 'hyp+'.
      hyp-: Intäkter från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 42 procent till ett belopp av 107,6 miljoner dollar.
      Detta är en parafras som inte stöds av källan. Den säger motsatsen till vad källan säger: intäkterna har ökat i de flesta kommunerna. Detta motsäger informationen i källan. Därför märks den som 'hyp-'.

      Nu, med den angivna källtexten nedan, generera 'hyp+' och 'hyp-' parafraser och ge resultatet i följande format: {answer_format}

      Källa: {source}
      <end_of_turn>

      Resultat:
      <start_of_turn>modell
      '''


    messages = [
      #{"role": "user", "content": "You are a text generator. You are specialized in paraphrasing texts."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=150,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
    generated_paraphrase = outputs[0]["generated_text"][len(prompt):]

    return generated_paraphrase



In [3]:
import json
import re

def extract_json_data(result_text):
    # Find the start and end indices of the JSON data
    start_index = result_text.find('{')
    end_index = result_text.find('}') + 1  # Add 1 to include the closing brace

    # Extract the JSON data from the string
    result_text = result_text[start_index:end_index]

    # Replace single quotes used as delimiters with double quotes

    result_text = result_text.replace("{'", '{"').replace("': '", '": "').replace("', '", '", "').replace("'}", '"}')

    # Convert the JSON data to a Python dictionary
    data_dict = json.loads(result_text)

    print('result_text:',result_text)

    return data_dict

In [ ]:
source = "My mom's food is better than my sister's."
result = generate_paraphrase(source)
extract_json_data(result)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'hyp+': 'My mom's food is better than my sister's food.', 'hyp-': 'My sister's food is better than my mom's food.'}

The source text states that "My mom's food is better than my sister's". Therefore, the paraphrase hypothesis hyp+ is supported by the source text. The paraphrase hypothesis hyp- is not supported by the source text.
result_text: {"hyp+": "My mom's food is better than my sister's food.", "hyp-": "My sister's food is better than my mom's food."}


{'hyp+': "My mom's food is better than my sister's food.",
 'hyp-': "My sister's food is better than my mom's food."}

## Generate dataset

In [4]:

import os
import numpy as np
import pandas as pd

def save_results(generate_paraphrase_func, prompt, df, csv_filename):
    # Split DataFrame into chunks of 10 rows
    chunks = np.array_split(df, len(df) // 10 if len(df) % 10 == 0 else len(df) // 10 + 1)

    for chunk in chunks:
        # Iterate through each row in the chunk
        for index, row in chunk.iterrows():
            source = row['source']

            try:
                # Generate paraphrase
                result = generate_paraphrase_func(source)
                extracted_hyp = extract_json_data(result)

                hyp_support = extracted_hyp['hyp+']
                print(hyp_support)
                hyp_hallu = extracted_hyp['hyp-']
                print(hyp_hallu)
                print('source:',source)

                # Update DataFrame with new values
                chunk.at[index, 'hyp+'] = hyp_support
                chunk.at[index, 'hyp-'] = hyp_hallu
                chunk.at[index, 'prompt'] = prompt
            except:
                print(f"Skipping row {index} due to error in data extraction.")
                continue

        # Append chunk to CSV file
        if not os.path.isfile(csv_filename):
            chunk.to_csv(csv_filename, mode='w', header=True, index=False)
        else:
            chunk.to_csv(csv_filename, mode='a', header=False, index=False)

        # Load CSV file
        df = pd.read_csv(csv_filename)
        display(df)

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_gemma/results_gemma_en_v1.csv"

prompt = '''
       <start_of_turn>user
       Given the src below, generate a paraphrase hypothesis  hyp+ that is supported by src and a second paraphrase hyp- that is not supported by src.
       Provide the result in the following format: {answer_format}
       Src: {source} <end_of_turn>
       Result:
       <start_of_turn>model
      '''

save_results(generate_paraphrase, prompt, data_df_en, csv_filename)

Gen text:  {'hyp+': 'The program has enabled the exchange of experiences and the pursuit of activities to raise the level of competence throughout Europe.', 'hyp-': 'The program has enabled us to support and encourage an exchange of experiences and to pursue activities to raise the level of competence throughout the world.'}
result_text: {"hyp+": "The program has enabled the exchange of experiences and the pursuit of activities to raise the level of competence throughout Europe.", "hyp-": "The program has enabled us to support and encourage an exchange of experiences and to pursue activities to raise the level of competence throughout the world."}
The program has enabled the exchange of experiences and the pursuit of activities to raise the level of competence throughout Europe.
The program has enabled us to support and encourage an exchange of experiences and to pursue activities to raise the level of competence throughout the world.
source: It has enabled us to support and encourage 

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'hyp+': 'The premiums for all varieties of leaf tobacco should be increased for the 1999, 2000 and 2001 harvests.', 'hyp-': 'The premiums for all varieties of leaf tobacco should be decreased for the 1999, 2000 and 2001 harvests.'}
result_text: {"hyp+": "The premiums for all varieties of leaf tobacco should be increased for the 1999, 2000 and 2001 harvests.", "hyp-": "The premiums for all varieties of leaf tobacco should be decreased for the 1999, 2000 and 2001 harvests."}
The premiums for all varieties of leaf tobacco should be increased for the 1999, 2000 and 2001 harvests.
The premiums for all varieties of leaf tobacco should be decreased for the 1999, 2000 and 2001 harvests.
source: Therefore, I am calling for an increase in the premiums for all varieties of leaf tobacco for the 1999, 2000 and 2001 harvests.
Gen text:  {'hyp+': 'A person may be prevented from coming near their victim when there is reason to fear that violent acts will be carried out again.', 'hyp-': 'A 

,id,source,hyp+,hyp-,prompt
0,0,It has enabled us to support and encourage an ...,The program has enabled the exchange of experi...,The program has enabled us to support and enco...,\n <start_of_turn>user\n Given the...
1,1,"Therefore, I am calling for an increase in the...",The premiums for all varieties of leaf tobacco...,The premiums for all varieties of leaf tobacco...,\n <start_of_turn>user\n Given the...
2,2,"In other words, a person may be prevented from...",A person may be prevented from coming near the...,A person may be prevented from coming near the...,\n <start_of_turn>user\n Given the...
3,3,None of my 34 amendments were adopted and my a...,The proposed administrative burden is ridiculo...,My amendments were adopted.,\n <start_of_turn>user\n Given the...
4,4,The final key feature of the report is its ori...,The report's orientation aims to protest again...,The report's orientation aims to promote the c...,\n <start_of_turn>user\n Given the...
5,5,It is high time an appropriate level of social...,An appropriate level of social protection shou...,The creation of an appropriate level of social...,\n <start_of_turn>user\n Given the...
6,6,"These sums, however, have still not produced t...",The substantial fall in the average earnings o...,The substantial fall in the average earnings o...,\n <start_of_turn>user\n Given the...
7,7,"Mr President, the approach adopted by the rapp...",NaN,NaN,NaN
8,8,"The Berlin summit, which will focus solely on ...",The Berlin summit will significantly impact th...,The Berlin summit will have little impact on t...,\n <start_of_turn>user\n Given the...
9,9,"As a result, I cannot promise the chairman of ...","The financial programme will be fully funded, ...",The financial programme will not be fully fund...,\n <start_of_turn>user\n Given the...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'hyp+': 'The pay and working conditions of navigation employees should be comparable to other occupations.', 'hyp-': 'The pay and working conditions of navigation employees should be significantly better than other occupations.'}
result_text: {"hyp+": "The pay and working conditions of navigation employees should be comparable to other occupations.", "hyp-": "The pay and working conditions of navigation employees should be significantly better than other occupations."}
The pay and working conditions of navigation employees should be comparable to other occupations.
The pay and working conditions of navigation employees should be significantly better than other occupations.
source: An important factor here is that the pay and other working conditions of those employed in navigation should be in line with those of other occupations.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'hyp+': 'The Conference would have produced more hot air if not for the chairmanship skills of the Deputy Prime Minister, John Prescott.', 'hyp-': 'The Conference would have produced less hot air if not for the chairmanship skills of the Deputy Prime Minister, John Prescott.'}
result_text: {"hyp+": "The Conference would have produced more hot air if not for the chairmanship skills of the Deputy Prime Minister, John Prescott.", "hyp-": "The Conference would have produced less hot air if not for the chairmanship skills of the Deputy Prime Minister, John Prescott."}
The Conference would have produced more hot air if not for the chairmanship skills of the Deputy Prime Minister, John Prescott.
The Conference would have produced less hot air if not for the chairmanship skills of the Deputy Prime Minister, John Prescott.
source: Indeed, without the chairmanship skills of the Deputy Prime Minister, John Prescott, the Conference would have produced more hot air than it prevented!
Ge

,id,source,hyp+,hyp-,prompt
0,0,It has enabled us to support and encourage an ...,The program has enabled the exchange of experi...,The program has enabled us to support and enco...,\n <start_of_turn>user\n Given the...
1,1,"Therefore, I am calling for an increase in the...",The premiums for all varieties of leaf tobacco...,The premiums for all varieties of leaf tobacco...,\n <start_of_turn>user\n Given the...
2,2,"In other words, a person may be prevented from...",A person may be prevented from coming near the...,A person may be prevented from coming near the...,\n <start_of_turn>user\n Given the...
3,3,None of my 34 amendments were adopted and my a...,The proposed administrative burden is ridiculo...,My amendments were adopted.,\n <start_of_turn>user\n Given the...
4,4,The final key feature of the report is its ori...,The report's orientation aims to protest again...,The report's orientation aims to promote the c...,\n <start_of_turn>user\n Given the...
...,...,...,...,...,...
114,114,I am referring to the current vote but in rela...,The speaker is referring to the current vote i...,The speaker is referring to a vote that has al...,\n <start_of_turn>user\n Given the...
115,115,When I became a Member of the European Parliam...,The Member of the European Parliament in 1994 ...,The Member of the European Parliament in 1994 ...,\n <start_of_turn>user\n Given the...
116,116,"Mr President, let me express my thanks for the...",The speaker is expressing gratitude and welcom...,The speaker is making a political statement.,\n <start_of_turn>user\n Given the...
117,117,We can replace 16 out of 626 Members of the Eu...,The European Parliament membership can be part...,The European Parliament membership cannot be c...,\n <start_of_turn>user\n Given the...


In [5]:
# update failed rows
def process_csv(file_path, generate_paraphrase_func, prompt):
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)

    # Initialize an empty list to store IDs
    missing_ids = []

    # Iterate over each row
    for index, row in df.iterrows():
        # Check if any of the specified columns are empty
        if pd.isna(row['prompt']) and pd.isna(row['hyp+']) and pd.isna(row['hyp-']):
            # Update the "source" and "hyp+" columns with the specified text
            source = row['source']

            missing_ids.append(row['id'])
            try:
              # Generate paraphrase
              result = generate_paraphrase_func(source)
              extracted_hyp = extract_json_data(result)

              hyp_support = extracted_hyp['hyp+']
              print("Hyp+ : ", hyp_support)
              hyp_hallu = extracted_hyp['hyp-']
              print("Hyp- : ", hyp_hallu)

              print('source: ',source)

              # Update DataFrame with new values
              df.at[index, 'hyp+'] = hyp_support
              df.at[index, 'hyp-'] = hyp_hallu
              df.at[index, 'prompt'] = prompt

              print("ADDED")
            except:
                print(f"Skipping row {index} due to error in data extraction.")
                continue

    rows_with_missing_ids = df[df['id'].isin(missing_ids)]

    display(rows_with_missing_ids)

    df.to_csv(file_path, header=True, index=False)

In [6]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_gemma/results_gemma_en_v1.csv"

prompt = '''
       <start_of_turn>user
       Given the src below, generate a paraphrase hypothesis  hyp+ that is supported by src and a second paraphrase hyp- that is not supported by src.
       Provide the result in the following format: {answer_format}
       Src: {source} <end_of_turn>
       Result:
       <start_of_turn>model
      '''

result = process_csv(csv_filename, generate_paraphrase, prompt)
result

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Gen text:  {'hyp+': "The approach adopted by the rapporteur to the Commission's 1999 annual economic report is comprehensive and sensible.", 'hyp-': "The approach adopted by the rapporteur to the Commission's 1999 annual economic report is outdated and ineffective."}
Skipping row 7 due to error in data extraction.
Gen text:  {'hyp+': "The death penalty against Greg Summers in Texas, USA, was carried out by Mrs d'Ancona on behalf of the PSE Group.", 'hyp-': "Mrs d'Ancona was not involved in the death penalty against Greg Summers."}

**Hyp+:** The death penalty against Greg Summers in Texas, USA, was carried out by Mrs d'Ancona on behalf of the PSE Group.

**Hyp-:** Mrs d'Ancona was not involved in the death penalty against Greg Summers.
Skipping row 12 due to error in data extraction.
Gen text:  {'hyp+': 'The Socialist Group supports the proposal enthusiastically after having pressed the Commission to present it.', 'hyp-': 'The Socialist Group is opposed to the proposal.'}
result_text: 

,id,source,hyp+,hyp-,prompt
7,7,"Mr President, the approach adopted by the rapp...",NaN,NaN,NaN
12,12,"B4-0188/99 by Mrs d\'Ancona, on behalf of the ...",NaN,NaN,NaN
31,31,Having pressed the Commission to present this ...,The Socialist Group supports the proposal enth...,The Socialist Group is opposed to the proposal.,\n <start_of_turn>user\n Given the...
34,34,The situation in India is appalling as 50 mill...,The situation in India is appalling as a large...,The situation in India is appalling as a large...,\n <start_of_turn>user\n Given the...
42,42,"For instance, in the summer it would perhaps b...",It would be possible to set up surveillance op...,The flimsy craft used to make the crossing wou...,\n <start_of_turn>user\n Given the...
59,59,"According to what I have heard, the ISO will a...",The ISO will have a label ready for the summer...,The ISO will not have a label ready for the su...,\n <start_of_turn>user\n Given the...
62,62,We should remember that the nuclear power stat...,The nuclear power station at Chernobyl was sai...,The nuclear power station at Chernobyl was sai...,\n <start_of_turn>user\n Given the...
76,76,People would think we were mad if we said tomo...,If drivers were allowed to take their cars on ...,If drivers were allowed to take their cars on ...,\n <start_of_turn>user\n Given the...
80,80,"Madam President, in yesterday\'s International...",NaN,NaN,NaN
81,81,The current status quo of a 12-mile limit for ...,The current 12-mile limit for the area is the ...,The current 12-mile limit for the area is not ...,\n <start_of_turn>user\n Given the...


In [7]:
# save a csv file in the task format

import pandas as pd

# Load the CSV file
df = pd.read_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_gemma/results_gemma_en_v1.csv')

# Select only the columns 'id', 'hyp+', and 'hyp-'
df = df[['id', 'hyp+', 'hyp-']]

# Save the new DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/final_gemma_en_v1.csv', index=False)

# Swedish

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_gemma/results_gemma_se_v1.csv"

prompt = '''
       <start_of_turn>användare
    Med tanke på källan nedan, generera en parafras-hypotes hyp+ som stöds av källan och en andra parafras hyp- som inte stöds av källan.
    Ge resultatet i följande format: {answer_format}
    Källa: {source}
    <end_of_turn>
    Resultat:
    <start_of_turn>modell
      '''

save_results(generate_paraphrase_se, prompt, data_df_se, csv_filename)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Gen text:  {'hyp+': 'Kvinnor kommer att möta högre bilförsäkringspremier på grund av den ökande bilkostnaden.', 'hyp-': 'Kvinnor kommer inte att möta högre bilförsäkringspremier på grund av den ökande bilkostnaden.'}

Källa: Kvinnor kommer att möta högre bilförsäkringspremier.

Resultat: Den första hypotesen stöds av källan, medan den andra hypotesen inte stöds av källan.
result_text: {"hyp+": "Kvinnor kommer att möta högre bilförsäkringspremier på grund av den ökande bilkostnaden.", "hyp-": "Kvinnor kommer inte att möta högre bilförsäkringspremier på grund av den ökande bilkostnaden."}
Kvinnor kommer att möta högre bilförsäkringspremier på grund av den ökande bilkostnaden.
Kvinnor kommer inte att möta högre bilförsäkringspremier på grund av den ökande bilkostnaden.
source: Kvinnor kommer att möta högre bilförsäkringspremier.
Gen text:  {'hyp+': 'Rörelseintäkterna är ökade jämfört med förra årets resultat.', 'hyp-': 'Rörelseintäkterna är inte ökade jämfört med förra årets resultat.'}



,id,source,hyp+,hyp-,prompt
0,0,Kvinnor kommer att möta högre bilförsäkringspr...,Kvinnor kommer att möta högre bilförsäkringspr...,Kvinnor kommer inte att möta högre bilförsäkri...,\n <start_of_turn>användare\n Med tan...
1,1,"Rörelseintäkterna var 1,45 miljarder dollar, e...",Rörelseintäkterna är ökade jämfört med förra å...,Rörelseintäkterna är inte ökade jämfört med fö...,\n <start_of_turn>användare\n Med tan...
2,2,"Mandela tillbaka på sjukhus i ""allvarligt men ...",Mandela tillbaka på sjukhus är i ett stabilt t...,Mandela tillbaka på sjukhus är i ett allvarlig...,\n <start_of_turn>användare\n Med tan...
3,3,Egypten fryser Muslimska brödraskapets tillgån...,Egypten fryser Muslimska brödraskapets tillgån...,Egypten fryser Muslimska brödraskapets tillgån...,\n <start_of_turn>användare\n Med tan...
4,4,"P-pillret, som de kallar ""polypiller"", skulle ...","P-pillret skulle innehålla acetylsalicylsyra, ...",P-pillret skulle innehålla ett annat läkemedel...,\n <start_of_turn>användare\n Med tan...
5,5,Beväpnad man dödar 6 i skottlossning mot Wisco...,Beväpnad man dödar 6 i skottlossning mot Wisco...,Beväpnad man dödar 6 i skottlossning mot Wisco...,\n <start_of_turn>användare\n Med tan...
6,6,Man som satte eld på sig själv på National Mal...,En person som satte eld på sig själv på Nation...,En person som satte eld på sig själv på Nation...,\n <start_of_turn>användare\n Med tan...
7,7,Romney tillkännager Paul Ryan som vicepresiden...,NaN,NaN,NaN
8,8,Läkare säger att en eller båda pojkarna kan dö...,En eller båda pojkarna kan dö,Verrera hjärnskador är möjliga om de överlever,\n <start_of_turn>användare\n Med tan...
9,9,Nordkorea säger att en amerikansk medborgare s...,Nordkorea säger att en amerikansk medborgare s...,Nordkorea kommer inte att ställa en amerikansk...,\n <start_of_turn>användare\n Med tan...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'hyp+': 'Källan indikerar att många personer dödades i luftangrepp på bageri i centrala Syrien.', 'hyp-': 'Källan indikerar inte att luftangrepp är en orodlig orsak till döden på bageri i centrala Syrien.'}

Källa: Dussintals dödade i luftangrepp på bageri i centrala Syrien.
result_text: {"hyp+": "Källan indikerar att många personer dödades i luftangrepp på bageri i centrala Syrien.", "hyp-": "Källan indikerar inte att luftangrepp är en orodlig orsak till döden på bageri i centrala Syrien."}
Källan indikerar att många personer dödades i luftangrepp på bageri i centrala Syrien.
Källan indikerar inte att luftangrepp är en orodlig orsak till döden på bageri i centrala Syrien.
source: Dussintals dödade i luftangrepp på bageri i centrala Syrien.


...

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'hyp+': 'Indiska trupper som redan kämpar mot rebeller i Kashmir är nu engagerade i en massiv kampanj för att förstöra vallmogrödor i regionen, vilket kommer att leda till en ökad stabilitet i Kashmir.', 'hyp-': 'Indiska trupper som redan kämpar mot rebeller i Kashmir är nu engagerade i en massiv kampanj för att förstöra vallmogrödor i regionen, vilket kommer att leda till en ökad instabilitet i Kashmir.'}
result_text: {"hyp+": "Indiska trupper som redan kämpar mot rebeller i Kashmir är nu engagerade i en massiv kampanj för att förstöra vallmogrödor i regionen, vilket kommer att leda till en ökad stabilitet i Kashmir.", "hyp-": "Indiska trupper som redan kämpar mot rebeller i Kashmir är nu engagerade i en massiv kampanj för att förstöra vallmogrödor i regionen, vilket kommer att leda till en ökad instabilitet i Kashmir."}
Indiska trupper som redan kämpar mot rebeller i Kashmir är nu engagerade i en massiv kampanj för att förstöra vallmogrödor i regionen, vilket kommer att l

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'hyp+': 'En explosion nära mållinjen i Boston Marathon kommer att orsaka en ökad säkerhetsrisk för participants.', 'hyp-': 'En explosion nära mållinjen i Boston Marathon kommer att orsaka en minskad säkerhetsrisk för participants.'}

Källa: Uppdaterad - Två explosioner nära mållinjen i Boston Marathon.

**Explanations:**

* **Hyp+:** Stöd av källan. En explosion nära mållinjen i Boston Marathon kommer att orsaka en ökad säkerhetsrisk för participants.
* **Hyp-:** Inte stöd av källan. En explosion nära mållinjen i Boston Marathon kommer att orsaka en minskad säkerhetsrisk för participants
result_text: {"hyp+": "En explosion nära mållinjen i Boston Marathon kommer att orsaka en ökad säkerhetsrisk för participants.", "hyp-": "En explosion nära mållinjen i Boston Marathon kommer att orsaka en minskad säkerhetsrisk för participants."}
En explosion nära mållinjen i Boston Marathon kommer att orsaka en ökad säkerhetsrisk för participants.
En explosion nära mållinjen i Boston Marat

,id,source,hyp+,hyp-,prompt
0,0,Kvinnor kommer att möta högre bilförsäkringspr...,Kvinnor kommer att möta högre bilförsäkringspr...,Kvinnor kommer inte att möta högre bilförsäkri...,\n <start_of_turn>användare\n Med tan...
1,1,"Rörelseintäkterna var 1,45 miljarder dollar, e...",Rörelseintäkterna är ökade jämfört med förra å...,Rörelseintäkterna är inte ökade jämfört med fö...,\n <start_of_turn>användare\n Med tan...
2,2,"Mandela tillbaka på sjukhus i ""allvarligt men ...",Mandela tillbaka på sjukhus är i ett stabilt t...,Mandela tillbaka på sjukhus är i ett allvarlig...,\n <start_of_turn>användare\n Med tan...
3,3,Egypten fryser Muslimska brödraskapets tillgån...,Egypten fryser Muslimska brödraskapets tillgån...,Egypten fryser Muslimska brödraskapets tillgån...,\n <start_of_turn>användare\n Med tan...
4,4,"P-pillret, som de kallar ""polypiller"", skulle ...","P-pillret skulle innehålla acetylsalicylsyra, ...",P-pillret skulle innehålla ett annat läkemedel...,\n <start_of_turn>användare\n Med tan...
...,...,...,...,...,...
71,71,Du kommer att komma på saker! Jag kommer att b...,"Källan stöds hypen att när man är uttråkad, då...",Källan stöds inte hypen att när man är uttråka...,\n <start_of_turn>användare\n Med tan...
72,72,Påven uppmanar till åtgärder mot klimatförändr...,Påven uppmanar till åtgärder mot klimatförändr...,Påven uppmanar till åtgärder mot klimatförändr...,\n <start_of_turn>användare\n Med tan...
73,73,Egyptens ledare inför undantagstillstånd i tre...,Egypten kommer att behålla undantagstillstånde...,Egypten kommer inte att behålla undantagstills...,\n <start_of_turn>användare\n Med tan...
74,74,En svartvit bild av en katt som ligger på en m...,En svartvit bild av en katt ligger stabilt på ...,En svartvit bild av en katt ligger överhängad ...,\n <start_of_turn>användare\n Med tan...


In [10]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_gemma/results_gemma_se_v1.csv"

prompt = '''
       <start_of_turn>användare
    Med tanke på källan nedan, generera en parafras-hypotes hyp+ som stöds av källan och en andra parafras hyp- som inte stöds av källan.
    Ge resultatet i följande format: {answer_format}
    Källa: {source}
    <end_of_turn>
    Resultat:
    <start_of_turn>modell
      '''

result = process_csv(csv_filename, generate_paraphrase_se, prompt)
result

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'hyp+': "Romney tillkännager Paul Ryan som vicepresidentkandidat kommer att öka rösterna för Republican Party.", 'hyp-': "Romney tillkännager Paul Ryan som vicepresidentkandidat kommer att minska rösterna för Republican Party."}

**Källa:** Romney tillkännager Paul Ryan som vicepresidentkandidat.
Skipping row 7 due to error in data extraction.
Gen text:  {'hyp+': "David Camerons seende av Andy Murrays förtjänares adelstitel är positiv.", 'hyp-': "David Camerons seende av Andy Murrays förtjänares adelstitel är negativa."}

**Källa:** Andy Murray förtjänaren adelstitel, säger David Cameron.
Skipping row 13 due to error in data extraction.
Gen text:  {'hyp+': "En äldre kvinna står i ett kök med två katter vid sina fötter är en situation där det finns en hög risk för fall.", 'hyp-': "En äldre kvinna står i ett kök med två katter vid sina fötter är en situation där det finns en låg risk för fall."}

**Källa:** En äldreMediabestanden står i ett kök med två katter vid sina fötter.

,id,source,hyp+,hyp-,prompt
7,7,Romney tillkännager Paul Ryan som vicepresiden...,NaN,NaN,NaN
13,13,"Andy Murray förtjänaren adelstitel, säger Davi...",NaN,NaN,NaN
19,19,En äldre kvinna står i ett kök med två katter ...,NaN,NaN,NaN
20,20,Baksidan av en stoppskylt med många klistermär...,En hög koncentration av klistermärken på en st...,En hög koncentration av klistermärken på en st...,\n <start_of_turn>användare\n Med tan...
27,27,"""När du gick över gränsen, kränkte du den kons...",NaN,NaN,NaN
28,28,"Snowden ser ""ingen chans"" för en rättvis rätte...",NaN,NaN,NaN
29,29,Anthony Weiner halkar ned till fjärde plats i ...,NaN,NaN,NaN
33,33,Helmand-provinsen är världens största opiumpro...,Helmand-provinsen är världens största opiumpro...,Opiumproduktion är en viktig faktor för västra...,\n <start_of_turn>användare\n Med tan...
41,41,"""Och nu är det vad han vill säga"", sa Alesha B...",NaN,NaN,NaN
58,58,Kevin Rudd svors in som Australiens premiärmin...,Kvanter av den totala tillgängliga resursen fö...,Den totala tillgängliga resursen för en nation...,\n <start_of_turn>användare\n Med tan...


In [ ]:
# Load the CSV file
df = pd.read_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_gemma/results_gemma_se_v1.csv')

# Select only the columns 'id', 'hyp+', and 'hyp-'
df = df[['id', 'hyp+', 'hyp-']]

# Save the new DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/final_gemma_se_v1.csv', index=False)

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_gemma/results_gemma_se_v2.csv"

prompt = '''
      <start_of_turn>användare
      Som en AI-modell är din uppgift att generera två parafraser baserade på den angivna källtexten. Den första parafrasen, märkt som 'hyp+', ska stödjas av källtexten. Den andra parafrasen, märkt som 'hyp-', ska inte stödjas av källtexten.
      Här är ett exempel för att illustrera detta:
      Källa: Intäkterna från mjukvarulicenser, ett mått som finansanalytiker följer noga, minskade med 21 procent till 107,6 miljoner dollar.
      hyp+: Intäkter från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 21 procent till ett belopp av 107,6 miljoner dollar.
      Detta är en parafras som stöds av källan. Den säger i princip samma sak som källan: intäkterna har minskat i de flesta kommunerna. Formuleringen är annorlunda, men betydelsen är densamma. Därför märks den som 'hyp+'.
      hyp-: Intäkter från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 42 procent till ett belopp av 107,6 miljoner dollar.
      Detta är en parafras som inte stöds av källan. Den säger motsatsen till vad källan säger: intäkterna har ökat i de flesta kommunerna. Detta motsäger informationen i källan. Därför märks den som 'hyp-'.
      Nu, med den angivna källtexten nedan, generera 'hyp+' och 'hyp-' parafraser och ge resultatet i följande format: {answer_format}
      Källa: {source}
      <end_of_turn>
      Resultat:
      <start_of_turn>modell
      '''

save_results(generate_paraphrase_se, prompt, data_df_se, csv_filename)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'hyp+': 'Kvinnor kommer att möta högre bilförsäkringspremier.', 'hyp-': 'Kvinnor kommer att möta lägre bilförsäkringspremier.'}
result_text: {"hyp+": "Kvinnor kommer att möta högre bilförsäkringspremier.", "hyp-": "Kvinnor kommer att möta lägre bilförsäkringspremier."}
Kvinnor kommer att möta högre bilförsäkringspremier.
Kvinnor kommer att möta lägre bilförsäkringspremier.
source: Kvinnor kommer att möta högre bilförsäkringspremier.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'hyp+': 'Rörelseintäkterna var 1,45 miljarder dollar, en ökning jämfört med förra årets resultat på 1,38 miljarder dollar.', 'hyp-': 'Rörelseintäkterna var 1,45 miljarder dollar, en minskning jämfört med förra årets resultat på 1,38 miljarder dollar.'}
result_text: {"hyp+": "Rörelseintäkterna var 1,45 miljarder dollar, en ökning jämfört med förra årets resultat på 1,38 miljarder dollar.", "hyp-": "Rörelseintäkterna var 1,45 miljarder dollar, en minskning jämfört med förra årets resultat på 1,38 miljarder dollar."}
Rörelseintäkterna var 1,45 miljarder dollar, en ökning jämfört med förra årets resultat på 1,38 miljarder dollar.
Rörelseintäkterna var 1,45 miljarder dollar, en minskning jämfört med förra årets resultat på 1,38 miljarder dollar.
source: Rörelseintäkterna var 1,45 miljarder dollar, en ökning jämfört med förra årets resultat på 1,38 miljarder dollar. 
Gen text:  {'hyp+': 'Mandela tillbaka på sjukhus i "allvarligt men stabilt" tillstånd.', 'hyp-': 'Mandela tillbaka

,id,source,hyp+,hyp-,prompt
0,0,Kvinnor kommer att möta högre bilförsäkringspr...,Kvinnor kommer att möta högre bilförsäkringspr...,Kvinnor kommer att möta lägre bilförsäkringspr...,\n <start_of_turn>användare\n Som en...
1,1,"Rörelseintäkterna var 1,45 miljarder dollar, e...","Rörelseintäkterna var 1,45 miljarder dollar, e...","Rörelseintäkterna var 1,45 miljarder dollar, e...",\n <start_of_turn>användare\n Som en...
2,2,"Mandela tillbaka på sjukhus i ""allvarligt men ...",NaN,NaN,NaN
3,3,Egypten fryser Muslimska brödraskapets tillgån...,Egypten fryser Muslimska brödraskapets tillgån...,Egypten fryser Muslimska brödraskapets tillgån...,\n <start_of_turn>användare\n Som en...
4,4,"P-pillret, som de kallar ""polypiller"", skulle ...",NaN,NaN,NaN
5,5,Beväpnad man dödar 6 i skottlossning mot Wisco...,Beväpnad man dödar 6 i skottlossning mot Wisco...,Beväpnad man dödar 6 i skottlossning mot Wisco...,\n <start_of_turn>användare\n Som en...
6,6,Man som satte eld på sig själv på National Mal...,Man som satte eld på sig själv på National Mal...,Man som satte eld på sig själv på National Mal...,\n <start_of_turn>användare\n Som en...
7,7,Romney tillkännager Paul Ryan som vicepresiden...,Romney tillkännager Paul Ryan som vicepresiden...,Romney tillkännager Paul Ryan som vicepresiden...,\n <start_of_turn>användare\n Som en...
8,8,Läkare säger att en eller båda pojkarna kan dö...,Läkare säger att en eller båda pojkarna kan dö...,Det är ovanligt att överleva en eller båda inj...,\n <start_of_turn>användare\n Som en...
9,9,Nordkorea säger att en amerikansk medborgare s...,Nordkorea säger att en amerikansk medborgare s...,Nordkorea säger att en amerikansk medborgare s...,\n <start_of_turn>användare\n Som en...


...

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'hyp+': 'Indiska trupper som redan kämpar mot rebeller i Kashmir är nu engagerade i en massiv kampanj för att förstöra vallmogrödor i regionen.', 'hyp-': 'Indiska trupper som redan kämpar mot rebeller i Kashmir är nu engagerade i en massiv kampanj för att förstöra vallmogrödor i regionen, vilket kommer att öka intäkterna i de flesta kommunerna.'}
result_text: {"hyp+": "Indiska trupper som redan kämpar mot rebeller i Kashmir är nu engagerade i en massiv kampanj för att förstöra vallmogrödor i regionen.", "hyp-": "Indiska trupper som redan kämpar mot rebeller i Kashmir är nu engagerade i en massiv kampanj för att förstöra vallmogrödor i regionen, vilket kommer att öka intäkterna i de flesta kommunerna."}
Indiska trupper som redan kämpar mot rebeller i Kashmir är nu engagerade i en massiv kampanj för att förstöra vallmogrödor i regionen.
Indiska trupper som redan kämpar mot rebeller i Kashmir är nu engagerade i en massiv kampanj för att förstöra vallmogrödor i regionen, vilket

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'hyp+': 'Uppdaterad - Två explosioner nära mållinjen i Boston Marathon. Intäkterna från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 21 procent till ett belopp av 107,6 miljoner dollar.', 'hyp-': 'Uppdaterad - Två explosioner nära mållinjen i Boston Marathon. Intäkterna från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 42 procent till ett belopp av 107,6 miljoner dollar.'}
result_text: {"hyp+": "Uppdaterad - Två explosioner nära mållinjen i Boston Marathon. Intäkterna från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 21 procent till ett belopp av 107,6 miljoner dollar.", "hyp-": "Uppdaterad - Två explosioner nära mållinjen i Boston Marathon. Intäkterna från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 42 procent till ett belopp av 107,6 miljoner dollar."}
Uppdaterad - Två explosion

,id,source,hyp+,hyp-,prompt
0,0,Kvinnor kommer att möta högre bilförsäkringspr...,Kvinnor kommer att möta högre bilförsäkringspr...,Kvinnor kommer att möta lägre bilförsäkringspr...,\n <start_of_turn>användare\n Som en...
1,1,"Rörelseintäkterna var 1,45 miljarder dollar, e...","Rörelseintäkterna var 1,45 miljarder dollar, e...","Rörelseintäkterna var 1,45 miljarder dollar, e...",\n <start_of_turn>användare\n Som en...
2,2,"Mandela tillbaka på sjukhus i ""allvarligt men ...",NaN,NaN,NaN
3,3,Egypten fryser Muslimska brödraskapets tillgån...,Egypten fryser Muslimska brödraskapets tillgån...,Egypten fryser Muslimska brödraskapets tillgån...,\n <start_of_turn>användare\n Som en...
4,4,"P-pillret, som de kallar ""polypiller"", skulle ...",NaN,NaN,NaN
...,...,...,...,...,...
71,71,Du kommer att komma på saker! Jag kommer att b...,Du kommer att komma på saker! Jag kommer att b...,Jag kommer att komma på saker! Jag kommer att ...,\n <start_of_turn>användare\n Som en...
72,72,Påven uppmanar till åtgärder mot klimatförändr...,Påven uppmanar till åtgärder mot klimatförändr...,Påven uppmanar till åtgärder mot klimatförändr...,\n <start_of_turn>användare\n Som en...
73,73,Egyptens ledare inför undantagstillstånd i tre...,Egyptens ledare inför undantagstillstånd i tre...,Egyptens ledare inför undantagstillstånd i tre...,\n <start_of_turn>användare\n Som en...
74,74,En svartvit bild av en katt som ligger på en m...,En svartvit bild av en katt som ligger på en m...,En svartvit bild av en katt som ligger på en m...,\n <start_of_turn>användare\n Som en...


In [ ]:
# Load the CSV file
df = pd.read_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_gemma/results_gemma_se_v2.csv')

# Select only the columns 'id', 'hyp+', and 'hyp-'
df = df[['id', 'hyp+', 'hyp-']]

# Save the new DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/final_gemma_se_v2.csv', index=False)